In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import os
import json

In [2]:
#read csv of iso codes

file= './Resources/iso_code.csv'

iso_code_df = pd.read_csv(file)

iso_code_df

,iso_code,total_vaccinations,people_fully_vaccinated_per_hundred
0,ABW,73360.0,18.84
1,AIA,6898.0,5.22
2,ALB,400064.0,0.02
3,AND,26414.0,6.06
4,ARE,10336214.0,38.79
...,...,...,...
116,TUR,21678461.0,9.84
117,URY,1695351.0,15.95
118,USA,232407669.0,28.93
119,ZAF,292623.0,0.49


In [3]:
#base url

#base_url= 'http://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL?format=json'


country_name= 'ALB'
variable= 'NY.GDP.MKTP.CD'
date= '2019'

base_url= f'http://api.worldbank.org/v2/country/{country_name}/indicator/{variable}?date={date}&format=json'


response= requests.get(base_url).json()

response

[{'page': 1,
  'pages': 1,
  'per_page': 50,
  'total': 1,
  'sourceid': '2',
  'sourcename': 'World Development Indicators',
  'lastupdated': '2021-04-26'},
 [{'indicator': {'id': 'NY.GDP.MKTP.CD', 'value': 'GDP (current US$)'},
   'country': {'id': 'AL', 'value': 'Albania'},
   'countryiso3code': 'ALB',
   'date': '2019',
   'value': 15279183289.9372,
   'unit': '',
   'obs_status': '',
   'decimal': 0}]]

In [4]:
iso_code_df.head()

,iso_code,total_vaccinations,people_fully_vaccinated_per_hundred
0,ABW,73360.0,18.84
1,AIA,6898.0,5.22
2,ALB,400064.0,0.02
3,AND,26414.0,6.06
4,ARE,10336214.0,38.79


In [5]:
#adding empty cloumns to df

#GDP Column
iso_code_df['GDP']= ''
iso_code_df

,iso_code,total_vaccinations,people_fully_vaccinated_per_hundred,GDP
0,ABW,73360.0,18.84,
1,AIA,6898.0,5.22,
2,ALB,400064.0,0.02,
3,AND,26414.0,6.06,
4,ARE,10336214.0,38.79,
...,...,...,...,...
116,TUR,21678461.0,9.84,
117,URY,1695351.0,15.95,
118,USA,232407669.0,28.93,
119,ZAF,292623.0,0.49,


In [6]:
test_df = iso_code_df.head(10)
test_df

,iso_code,total_vaccinations,people_fully_vaccinated_per_hundred,GDP
0,ABW,73360.0,18.84,
1,AIA,6898.0,5.22,
2,ALB,400064.0,0.02,
3,AND,26414.0,6.06,
4,ARE,10336214.0,38.79,
5,ARG,7386650.0,1.95,
6,AUT,2863389.0,8.73,
7,AZE,1440243.0,4.87,
8,BEL,3563371.0,6.60,
9,BGD,7954176.0,1.31,


In [7]:
gdp_list= []

variable= 'NY.GDP.MKTP.CD'
date= '2019'


for index, iso in enumerate(test_df['iso_code']):
    
    try:
        
        base_url= f'http://api.worldbank.org/v2/country/{iso}/indicator/{variable}?date={date}&format=json'
        #base_url= f'http://api.worldbank.org/v2/country/{iso}/indicator/{variable}?format=json'
    
        response= requests.get(base_url).json()
        
        #print(response)
    
        response_iso= response[1][0]['value']
        print(f'{iso}s GDP value is {response_iso}')
        print(base_url)
        
    except: 
        print('Key/Index Error')
        
    
    
    




ABWs GDP value is None
http://api.worldbank.org/v2/country/ABW/indicator/NY.GDP.MKTP.CD?date=2019&format=json
Key/Index Error
ALBs GDP value is 15279183289.9372
http://api.worldbank.org/v2/country/ALB/indicator/NY.GDP.MKTP.CD?date=2019&format=json
ANDs GDP value is 3154057987.23833
http://api.worldbank.org/v2/country/AND/indicator/NY.GDP.MKTP.CD?date=2019&format=json
AREs GDP value is 421142267937.65
http://api.worldbank.org/v2/country/ARE/indicator/NY.GDP.MKTP.CD?date=2019&format=json
ARGs GDP value is 445445177459.426
http://api.worldbank.org/v2/country/ARG/indicator/NY.GDP.MKTP.CD?date=2019&format=json
AUTs GDP value is 445075391688.156
http://api.worldbank.org/v2/country/AUT/indicator/NY.GDP.MKTP.CD?date=2019&format=json
AZEs GDP value is 48047647058.8235
http://api.worldbank.org/v2/country/AZE/indicator/NY.GDP.MKTP.CD?date=2019&format=json
BELs GDP value is 533097455834.509
http://api.worldbank.org/v2/country/BEL/indicator/NY.GDP.MKTP.CD?date=2019&format=json
BGDs GDP value is 302